In [2]:
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
import unicodedata


ModuleNotFoundError: No module named 'requests'

In [ ]:
def generate_variations(word,language='ancient greek',show=False):
        # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
        #DECLINES WORDS
        #test words
        #'Ῥωμαῖος' romans
        #'μισώ' hate
        #'εἰμί' are to be
        #word = word.lower()

        #https://en.wiktionary.org/wiki/Ῥωμαῖος
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')




        #remove unwanted classes
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        for tag in soup.find_all('table',{'class':'audiotable'}):
                tag.decompose()
        for tag in soup.find_all('sup'):
                tag.decompose()

        #HEADERS ---------------------------
        #find all tables
        tables =soup.find_all('table')
        part_of_speech_h3 =[]
        h4 =[] # inflection/declension
        nav=[] #verb tense, noun first second third delcension
        gender =None

        #DATAFRAMES----------------------------------

        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                
                header = tables[i].find_previous('h2').text.replace('[edit]','').lower().strip()

                #GENDER
                gender = tables[i].find_previous(class_="gender")
                if(gender != None):
                        gender = gender.find_next("abbr").text
                elif (gender ==None):
                        gender = "n"


                #if(header =='greek'):
                        # if modern greek ignore
                        #header = ''
                #if(header =='ancient greek'):
                        #header = 'greek'
                if(header != language):
                        part_of_speech_h3.append('')
                        h4.append('')
                        nav.append('')
                        continue
                if(tables[i].find_previous('h3')):
                        part_of_speech_h3.append(tables[i].find_previous('h3').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        part_of_speech_h3.append('')
                if(tables[i].find_previous('h4')):
                        h4.append(tables[i].find_previous('h4').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        h4.append('')
                if(tables[i].find_previous('div',{'class':'NavHead'})):
                        nav.append(tables[i].find_previous('div',{'class':'NavHead'}).text
                        .replace('hide ▲','')
                        .replace('\"','').replace('/','-').replace(':','-').strip().lower())
                        #  : / mess up the file path so you need to remove them
                else:
                        nav.append('')
                ##display(df)
                ##CREATE A FOLDER
                root_dir = f'{header.replace(" ", "_")}_variations'
                os.makedirs(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/',exist_ok=True)
                print(f'{root_dir}/{part_of_speech_h3[i]}/{word}/')
                ##os.makedirs(nav[i],exist_ok=True)

                #print(h3[i])
                #print(h4[i])
                #print(nav[i])
                #print('WRITE TO FILE--------------------------------')

                df=dataframes[i]

             
 
                # for appending df2 at the end of df1
                df.loc[len(df)]= gender
                df.iat[-1,0] = "gender"

                df=df.dropna(how='all',axis=0)



                df=df.replace({'➤':''}, regex=True)
                df=df.replace({'•':''}, regex=True)
                df=df.replace({'\[':'','\]':'','\{':'','\}':''}, regex=True)
                df=df.replace({'—':' '}, regex=True)
                #df=df.replace({'/':' ',r'\\':' '}, regex=True)
                df=df.replace(r"\([^)]*\)","",regex=True)



                # make lowercase
                df = df.applymap(lambda s: s.lower() if type(s) == str else s)

                #df = df.applymap(lambda s:strip_accents_and_lowercase(s) if type(s) == str else s)
                df = df.applymap(lambda s: s.split(',', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('-', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('+', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('/', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('\\', 1)[0] if type(s) == str else s)

                df= df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


                df = df.replace(r"\s\s+"," ",regex=True)
              
                # NAN ARE REPALCED BY SPACE CHARACTER
                df.to_csv(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/{nav[i]}_{h4[i]}.csv',index=False)

                return df

In [ ]:
class GreekWord:
    def __init__(self, id, text, bare_text, sequence_num, morph_code, base_form, bare_base_form, definition):
        self.id = id
        self.text = text
        self.bare_text = bare_text
        self.sequence_num = sequence_num
        self.morph_code = morph_code
        self.base_form = base_form
        self.bare_base_form = bare_base_form
        self.definition = definition

In [ ]:
def generate_all_forms(word):
    generate_variations(word) #generates the csv table dataframe from wiktionary
    #1. read the csv
    #2. take in block of all with base word
    #3. load all words into custom object, hashmap from morph code to object
    #4. scrape the chart, take the chart and figure out what morph codes are missing
    #5. generate object with all the words from the missing morph codes
    #6. edit csv in place or edit to dataframe to spit into csv
    #7. termination function that stores current location

In [ ]:
import pandas as pd

def update_csv_with_wiktionary(csv_path, base_word):
    # 1: Read the CSV
    df = pd.read_csv(csv_path)

    # 2: Scrape the chart to obtain all morph codes
    scraped_dataframe = generate_variations(base_word)  # Assuming generate_variations returns the DataFrame
    all_morph_codes = generate_all_morph_codes(base_word, csv_path, scraped_dataframe)

    # 3: Take in block of all words with the base word
    base_word_block = df[df['base_form'] == base_word]

    # 4: Load all words into custom object (hashmap from morph code to object)
    greek_word = {}
    for index, row in base_word_block.iterrows():
        morph_code = row['morph_code']
        word = GreekWord(
            id=row['id'],
            text=row['text'],
            bare_text=row['bare_text'],
            sequence_num=row['sequence_num'],
            morph_code=row['morph_code'],
            base_form=row['base_form'],
            bare_base_form=row['bare_base_form'],
            definition=row['definition']
        )
        greek_word[morph_code] = word

    # 5: Determine missing morph codes
    missing_morph_codes = []
    for morph_code in all_morph_codes:
        if morph_code not in greek_word:
            missing_morph_codes.append(morph_code)

    # 6: Generate object with all words from the missing morph codes
    missing_words = []
    for morph_code in missing_morph_codes:
        # Call generate_variations to get word details from Wiktionary
        word_details_from_wiktionary = generate_variations(base_word, morph_code)
        word = GreekWord(
            id=word_details_from_wiktionary['id'],
            text=word_details_from_wiktionary['text'],
            bare_text=word_details_from_wiktionary['bare_text'],
            sequence_num=word_details_from_wiktionary['sequence_num'],
            morph_code=morph_code,
            base_form=word_details_from_wiktionary['base_form'],
            bare_base_form=word_details_from_wiktionary['bare_base_form'],
            definition=word_details_from_wiktionary['definition']
        )
        greek_word[morph_code] = word
        missing_words.append(word)

    # 7: Edit DataFrame to export to CSV
    for word in missing_words:
        df = df.append(word.__dict__, ignore_index=True)

    # 8: Termination function that stores current location
    updated_csv_path = csv_path.replace('.csv', '_updated.csv')
    df.to_csv(updated_csv_path, index=False)

    return updated_csv_path

In [ ]:
def generate_all_morph_codes(word, original_csv_path, dataframe):
    # Read the original CSV containing Greek forms
    original_df = pd.read_csv(original_csv_path)

    # Extract all Greek forms from the DataFrame
    greek_forms = set(original_df['text'])

    # Compare with forms of the base word in the generated DataFrame
    base_word_forms = set(dataframe['text'])
    missing_forms = greek_forms - base_word_forms

    # Generate new entries for missing forms
    new_entries = []
    for form in missing_forms:
        # Retrieve the details of the missing form from the original CSV
        missing_entry = original_df[original_df['text'] == form].iloc[0]

        # Generate morph code based on the format provided in the chart
        morph_code = generate_morph_code(dataframe, form)

        new_entry = {
            'id': '',  # You may generate an appropriate ID here if needed
            'text': missing_entry['text'],
            'bare_text': missing_entry['bare_text'],
            'sequence_num': missing_entry['sequence_num'],
            'morph_code': morph_code,
            'base_form': word,
            'bare_base_form': missing_entry['bare_base_form'],
            'definition': missing_entry['definition']
        }
        new_entries.append(new_entry)

    return new_entries



In [ ]:
def generate_morph_code(dataframe, form):
    # Initialize variables to store morph code components
    part_of_speech = ''  # First letter of morph code
    number = ''           # Third letter of morph code
    gender = ''           # Seventh letter of morph code
    case = ''             # Eighth letter of morph code

    # Iterate through the DataFrame rows to find the form and extract its morph code components
    for index, row in dataframe.iterrows():
        # Check if the current row contains the form we are looking for
        if row['text'] == form:
            # Extract morph code components based on the row's position and content
            # For example, if it's an adjective in singular masculine nominative case, set the corresponding variables
            if row['1'] == 'adjective':
                part_of_speech = 'a'  # Assuming 'a' for adjective
                # Check the number (singular, dual, plural) and set the number variable accordingly
                # For example:
                # if row['2'] == 'singular':
                #     number = 's'
                # elif row['2'] == 'dual':
                #     number = 'd'
                # else:
                #     number = 'p'
                # Similarly, check other columns (gender, case) and set the corresponding variables
                # For example:
                # if row['7'] == 'masculine':
                #     gender = 'm'
                # elif row['7'] == 'feminine':
                #     gender = 'f'
                # else:
                #     gender = 'n'
                # Similarly for the case:
                # if row['8'] == 'nominative':
                #     case = 'n'
                # elif row['8'] == 'genitive':
                #     case = 'g'
                # and so on...
            # Add other conditions for different parts of speech and their variations

    # Combine morph code components to form the complete morph code
    morph_code = part_of_speech + '-' + number + '---' + gender + case + '-'

    return morph_code


In [ ]:
# Idea: Compile all of the morph codes of a single word from the Perseus database
# Then take this list of morph codes and compare it to a list of generate morph codes
# For the same word in the Wiktionary database. If the lists don't match, then we are missing forms
# and we can add them from the Wiktionary database

# Things to do:
# Get all forms of words of all base words from Wiktionary using generate_variations function
# 1. Function to scrape Wiktionary for variations of a word
# 2. Function to add variations to a CSV file (Wiktionary Dataframe)
# 3. Function to compare get_perseus_morphcodes and get_wiktionary_morphcodes
# 4. Function to add missing forms to Perseus Dataframe 
# 5. Celebrate 🎉

# Wiktionary CSV



# Function to get all morph codes for a given word from the Perseus database
def get_perseus_morphcodes(perseus_df_path, base_word):

    # Read the Perseus CSV file
    perseus_df = pd.read_csv(perseus_df_path)

    morph_codes = []
    # Extract all morph codes for the given word
    for index, row in perseus_df.iterrows():
        if row['base_form'] == base_word:
            morph_codes.append(row['morph_code'])


In [7]:
# Function to get all morph codes for a given word from the Wiktionary database
def get_wiktionary_morphcodes(wiktionary_df_path, base_word):
    # Read the Wiktionary DataFrame
    wiktionary_df = pd.read_csv(wiktionary_df_path)

    morph_codes = []
    # Extract all morph codes for the given word
    for index, row in wiktionary_df.iterrows():
        if row['base_form'] == base_word:
            morph_codes.append(row['morph_code'])

    return morph_codes

In [ ]:
def scrape_gen_varitions(base_word):
    # Jacob and Guhan
    # Scrapes the wiktionary page for the word and all of its variations

    # I'm thinking we put this in a map? Where the key is the plurality/grammatical case and the value is the form
    # Ex. Singular Nominative = SN -> map['SN'] = 'Singualar Nominative Form'

    # Set up the url to scrape
    url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
    soup = bs(url.text, 'html.parser')

    # Initialize storage map
    variations = {}

    # Remove unwanted html classes and stuff
    for tag in soup.find_all('span',{'class':'tr Latn'}):
            tag.decompose()
    for tag in soup.find_all('table',{'class':'audiotable'}):
            tag.decompose()
    for tag in soup.find_all('sup'):
            tag.decompose()

    tables = soup.find_all('table') 
    part_of_speech_h3 = [] # Stores part of speech headers (noun, verb, etc.)
    declension_h4 = [] # Stores inflection/declension headers (first, second, etc.)
    nav = [] # Stores something idk it was in his code
    gender = None # Init gender by default to None (gonna have it check for a neutral case, then check for f and m) (Also no clue if we need this his code is pretty messy lol)

    # Not sure about this yet still working on it
    #for i, table in enumerate(tables):
        


    




    return None






In [9]:
# possible variation w/ input path and output file path 
# takes in all of greek_words.csv file and returns them to the output path 
# iterate throught the index, row in the current Dataframe (in this case it would be greek_words.csv)
# and add them to the csv superset 
# %pip install pandas
# ignore, issues w/ local system
import pandas as pd
def add_variations_wiktionaryDataframes(csv_input_path, csv_output_path):
    # Johnny

    try: 
        # input csv read in 
        currDataframe = pd.read_csv(csv_input_path, header=0, on_bad_lines='skip', engine='python')
        currDataframe.columns = currDataframe.columns.str.strip()  # get rid of extra whitespacing issues like \t or bnrt
        print("this is the currentDataFrame header ", currDataframe.columns)
        # new df for output w same structure as original greek_words.csv file
        outputDataframe = pd.DataFrame(columns=['id', 'text', 'bare_text', 'sequence_num', 'morph_code',
                                                'base_form', 'bare_base_form', 'definition'])

        # just move the row from the greek_words.csv to the output df
        for index, row in currDataframe.iterrows():
            newRow = pd.DataFrame([{
                'id': row['id'],
                'text': row['text'],
                'bare_text': row['bare_text'],
                'sequence_num': row['sequence_num'],
                'morph_code': row['morph_code'],
                'base_form': row['base_form'],
                'bare_base_form': row['bare_base_form'],
                'definition': row['definition']
            }])
            # append the new row to the output df
            # append became deprecated 
            outputDataframe = pd.concat([outputDataframe, newRow], ignore_index=True)

        # write the output df to the new csv file
        outputDataframe.to_csv(csv_output_path, index=False)
        print(f"Data has been transferred to the {csv_output_path}")

    except Exception as e:
        print("Error occurred: ", e)



inputPath = "greek_dictionary/temp_input.csv"
outputPath = "greek_dictionary/temp_output.csv"

add_variations_wiktionaryDataframes(inputPath, outputPath)


this is the currentDataFrame header  Index(['id', 'text', 'bare_text', 'sequence_num', 'morph_code', 'base_form',
       'bare_base_form', 'definition'],
      dtype='object')
Data has been transferred to the greek_dictionary/temp_output.csv


In [ ]:
def compare_morph_codes(perseus_morph_codes, wiktionary_morph_codes, return_indices=False):
    # returns values by default but returns indices if specified
    # Krish - compare and find differences
    if return_indices:
        # find indices unique to perseus list
        unique_to_perseus = [i for i, x in enumerate(perseus_morph_codes) if x not in wiktionary_morph_codes]
        # find indices unique to wiktionary list
        unique_to_wiktionary = [i for i, x in enumerate(wiktionary_morph_codes) if x not in perseus_morph_codes]
    else:
        # find values unique to perseus list
        unique_to_perseus = [x for x in perseus_morph_codes if x not in wiktionary_morph_codes]
        # find values unique to wiktionary list
        unique_to_wiktionary = [x for x in wiktionary_morph_codes if x not in perseus_morph_codes]
    result = {"perseus" : unique_to_perseus, "wiktionary" : unique_to_wiktionary}
    # returns a dictionary with unique values/indices in each respective list
    return result
